<a href="https://colab.research.google.com/github/Sp-supriya/HCDS-Projects/blob/main/Project_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# **Organize and host webinars or workshops related to the product’s technical features**


In [ ]:
pip install pandas pytz

In [ ]:
!pip install pandas pytz uuid email-validator python-dateutil

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.6/313.6 kB 8.9 MB/s eta 0:00:00
  Created wheel for uuid: filename=uuid-1.30-py3-none-any.whl size=6478 sha256=b05c12993fe08ab1eb1b8d91f2f0449813683bafbc71a87507db8f534baaa230
  Stored in directory: /root/.cache/pip/wheels/ed/08/9e/f0a977dfe55051a07e21af89200125d65f1efa60cbac61ed88
Successfully built uuid


In [ ]:
import pandas as pd
import pytz
import uuid
from datetime import datetime, timedelta
import sqlite3
from enum import Enum
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from dataclasses import dataclass
from typing import List, Dict, Optional

In [ ]:
from datetime import datetime, timedelta
import sqlite3
from enum import Enum
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
import pandas as pd
import uuid
import pytz
from dataclasses import dataclass
from typing import List, Dict, Optional

class WebinarStatus(Enum):
    SCHEDULED = "Scheduled"
    LIVE = "Live"
    COMPLETED = "Completed"
    CANCELLED = "Cancelled"

class WebinarType(Enum):
    PRODUCT_DEMO = "Product Demo"
    TECHNICAL_DEEP_DIVE = "Technical Deep Dive"
    HANDS_ON_WORKSHOP = "Hands-on Workshop"
    Q_AND_A = "Q&A Session"
    FEATURE_SHOWCASE = "Feature Showcase"

@dataclass
class Presenter:
    id: int
    name: str
    title: str
    bio: str
    expertise: List[str]
    email: str

@dataclass
class WebinarContent:
    title: str
    description: str
    agenda: List[str]
    prerequisites: List[str]
    resources: Dict[str, str]  # Resource name -> URL
    presentation_url: Optional[str] = None
    recording_url: Optional[str] = None

class WebinarManagementSystem:
    def __init__(self, db_name="webinar_system.db"):
        self.conn = sqlite3.connect(db_name)
        self.setup_database()

    def setup_database(self):
        cursor = self.conn.cursor()

        # Webinars table
        cursor.execute('''
        CREATE TABLE IF NOT EXISTS webinars (
            id TEXT PRIMARY KEY,
            title TEXT,
            type TEXT,
            description TEXT,
            start_time DATETIME,
            duration_minutes INTEGER,
            max_participants INTEGER,
            status TEXT,
            zoom_meeting_id TEXT,
            zoom_meeting_password TEXT,
            recording_url TEXT,
            presentation_url TEXT
        )''')

        # Presenters table
        cursor.execute('''
        CREATE TABLE IF NOT EXISTS presenters (
            id INTEGER PRIMARY KEY,
            name TEXT,
            title TEXT,
            bio TEXT,
            expertise TEXT,
            email TEXT
        )''')

        # Participants table
        cursor.execute('''
        CREATE TABLE IF NOT EXISTS participants (
            id INTEGER PRIMARY KEY,
            email TEXT,
            name TEXT,
            company TEXT,
            job_title TEXT,
            registration_date DATETIME
        )''')

        # Webinar-Presenter relationship
        cursor.execute('''
        CREATE TABLE IF NOT EXISTS webinar_presenters (
            webinar_id TEXT,
            presenter_id INTEGER,
            FOREIGN KEY (webinar_id) REFERENCES webinars (id),
            FOREIGN KEY (presenter_id) REFERENCES presenters (id)
        )''')

        # Webinar-Participant relationship
        cursor.execute('''
        CREATE TABLE IF NOT EXISTS webinar_participants (
            webinar_id TEXT,
            participant_id INTEGER,
            registration_date DATETIME,
            attended BOOLEAN,
            feedback_submitted BOOLEAN,
            FOREIGN KEY (webinar_id) REFERENCES webinars (id),
            FOREIGN KEY (participant_id) REFERENCES participants (id)
        )''')

        # Feedback table
        cursor.execute('''
        CREATE TABLE IF NOT EXISTS feedback (
            id INTEGER PRIMARY KEY,
            webinar_id TEXT,
            participant_id INTEGER,
            rating INTEGER,
            comments TEXT,
            submission_date DATETIME,
            FOREIGN KEY (webinar_id) REFERENCES webinars (id),
            FOREIGN KEY (participant_id) REFERENCES participants (id)
        )''')

        # Resources table
        cursor.execute('''
        CREATE TABLE IF NOT EXISTS resources (
            id INTEGER PRIMARY KEY,
            webinar_id TEXT,
            name TEXT,
            url TEXT,
            type TEXT,
            FOREIGN KEY (webinar_id) REFERENCES webinars (id)
        )''')

        self.conn.commit()

    def create_webinar(self, content: WebinarContent, webinar_type: WebinarType,
                      start_time: datetime, duration_minutes: int,
                      max_participants: int, presenters: List[Presenter]) -> str:
        webinar_id = str(uuid.uuid4())
        cursor = self.conn.cursor()

        # Insert webinar
        cursor.execute('''
        INSERT INTO webinars (
            id, title, type, description, start_time,
            duration_minutes, max_participants, status
        ) VALUES (?, ?, ?, ?, ?, ?, ?, ?)
        ''', (webinar_id, content.title, webinar_type.value, content.description,
              start_time, duration_minutes, max_participants,
              WebinarStatus.SCHEDULED.value))

        # Insert presenters if they don't exist and create relationships
        for presenter in presenters:
            cursor.execute('''
            INSERT OR IGNORE INTO presenters (
                name, title, bio, expertise, email
            ) VALUES (?, ?, ?, ?, ?)
            ''', (presenter.name, presenter.title, presenter.bio,
                  ','.join(presenter.expertise), presenter.email))

            cursor.execute('''
            INSERT INTO webinar_presenters (webinar_id, presenter_id)
            VALUES (?, (SELECT id FROM presenters WHERE email = ?))
            ''', (webinar_id, presenter.email))

        # Insert resources
        for name, url in content.resources.items():
            cursor.execute('''
            INSERT INTO resources (webinar_id, name, url)
            VALUES (?, ?, ?)
            ''', (webinar_id, name, url))

        self.conn.commit()
        return webinar_id

    def register_participant(self, webinar_id: str, email: str, name: str,
                           company: str, job_title: str) -> bool:
        cursor = self.conn.cursor()

        # Check if webinar is full
        cursor.execute('''
        SELECT w.max_participants, COUNT(wp.participant_id) as current_participants
        FROM webinars w
        LEFT JOIN webinar_participants wp ON w.id = wp.webinar_id
        WHERE w.id = ?
        GROUP BY w.id
        ''', (webinar_id,))

        result = cursor.fetchone()
        if result and result[1] >= result[0]:
            return False

        # Insert participant if not exists
        cursor.execute('''
        INSERT OR IGNORE INTO participants (email, name, company, job_title, registration_date)
        VALUES (?, ?, ?, ?, ?)
        ''', (email, name, company, job_title, datetime.now()))

        # Create webinar-participant relationship
        cursor.execute('''
        INSERT INTO webinar_participants (
            webinar_id, participant_id, registration_date, attended, feedback_submitted
        ) VALUES (?, (SELECT id FROM participants WHERE email = ?), ?, FALSE, FALSE)
        ''', (webinar_id, email, datetime.now()))

        self.conn.commit()
        self._send_registration_confirmation(email, webinar_id)
        return True

    def _send_registration_confirmation(self, email: str, webinar_id: str):
        # Get webinar details
        cursor = self.conn.cursor()
        cursor.execute('''
        SELECT title, start_time, duration_minutes
        FROM webinars
        WHERE id = ?
        ''', (webinar_id,))
        webinar = cursor.fetchone()

        # Email configuration (replace with your SMTP settings)
        smtp_config = {
            'server': 'smtp.example.com',
            'port': 587,
            'username': 'your_email@example.com',
            'password': 'your_password'
        }

        msg = MIMEMultipart()
        msg['Subject'] = f'Registration Confirmation: {webinar[0]}'
        msg['From'] = smtp_config['username']
        msg['To'] = email

        body = f"""
        Thank you for registering for {webinar[0]}!

        Date: {webinar[1]}
        Duration: {webinar[2]} minutes

        You will receive the webinar link 24 hours before the event.
        """

        msg.attach(MIMEText(body, 'plain'))

        # Uncomment to enable actual email sending
        # with smtplib.SMTP(smtp_config['server'], smtp_config['port']) as server:
        #     server.starttls()
        #     server.login(smtp_config['username'], smtp_config['password'])
        #     server.send_message(msg)

    def update_webinar_status(self, webinar_id: str, status: WebinarStatus):
        cursor = self.conn.cursor()
        cursor.execute('''
        UPDATE webinars
        SET status = ?
        WHERE id = ?
        ''', (status.value, webinar_id))
        self.conn.commit()

    def record_attendance(self, webinar_id: str, participant_email: str):
        cursor = self.conn.cursor()
        cursor.execute('''
        UPDATE webinar_participants
        SET attended = TRUE
        WHERE webinar_id = ? AND
              participant_id = (SELECT id FROM participants WHERE email = ?)
        ''', (webinar_id, participant_email))
        self.conn.commit()

    def submit_feedback(self, webinar_id: str, participant_email: str,
                       rating: int, comments: str):
        cursor = self.conn.cursor()
        cursor.execute('''
        INSERT INTO feedback (
            webinar_id, participant_id, rating, comments, submission_date
        ) VALUES (
            ?, (SELECT id FROM participants WHERE email = ?), ?, ?, ?
        )
        ''', (webinar_id, participant_email, rating, comments, datetime.now()))

        cursor.execute('''
        UPDATE webinar_participants
        SET feedback_submitted = TRUE
        WHERE webinar_id = ? AND
              participant_id = (SELECT id FROM participants WHERE email = ?)
        ''', (webinar_id, participant_email))

        self.conn.commit()

    def generate_webinar_report(self, webinar_id: str) -> Dict:
        cursor = self.conn.cursor()

        # Get webinar details
        cursor.execute('''
        SELECT w.*,
               COUNT(DISTINCT wp.participant_id) as registered_participants,
               COUNT(DISTINCT CASE WHEN wp.attended THEN wp.participant_id END) as attended_participants,
               AVG(f.rating) as average_rating
        FROM webinars w
        LEFT JOIN webinar_participants wp ON w.id = wp.webinar_id
        LEFT JOIN feedback f ON w.id = f.webinar_id
        WHERE w.id = ?
        GROUP BY w.id
        ''', (webinar_id,))

        webinar_data = cursor.fetchone()

        # Get participant demographics
        cursor.execute('''
        SELECT p.company, COUNT(*) as count
        FROM webinar_participants wp
        JOIN participants p ON wp.participant_id = p.id
        WHERE wp.webinar_id = ?
        GROUP BY p.company
        ''', (webinar_id,))

        demographics = cursor.fetchall()

        return {
            'webinar_info': {
                'title': webinar_data[1],
                'type': webinar_data[2],
                'status': webinar_data[7]
            },
            'participation': {
                'registered': webinar_data[-3],
                'attended': webinar_data[-2],
                'attendance_rate': (webinar_data[-2] / webinar_data[-3] * 100
                                  if webinar_data[-3] > 0 else 0)
            },
            'feedback': {
                'average_rating': webinar_data[-1],
                'demographics': {company: count for company, count in demographics}
            }
        }

    def get_upcoming_webinars(self) -> List[Dict]:
        cursor = self.conn.cursor()
        cursor.execute('''
        SELECT w.*, COUNT(wp.participant_id) as registered_participants
        FROM webinars w
        LEFT JOIN webinar_participants wp ON w.id = wp.webinar_id
        WHERE w.start_time > DATETIME('now')
        GROUP BY w.id
        ORDER BY w.start_time
        ''')

        columns = ['id', 'title', 'type', 'description', 'start_time',
                  'duration_minutes', 'max_participants', 'status',
                  'zoom_meeting_id', 'zoom_meeting_password', 'recording_url',
                  'presentation_url', 'registered_participants']

        return [dict(zip(columns, row)) for row in cursor.fetchall()]

    def close(self):
        self.conn.close()

# Example usage
def main():
    system = WebinarManagementSystem()

    # Create a webinar
    content = WebinarContent(
        title="Advanced API Integration Techniques",
        description="Deep dive into API integration best practices",
        agenda=["Introduction", "Authentication", "Error Handling", "Best Practices"],
        prerequisites=["Basic API knowledge", "Python programming"],
        resources={"Documentation": "https://example.com/docs"}
    )

    presenter = Presenter(
        id=1,
        name="John Doe",
        title="Senior Technical Architect",
        bio="15 years of API development experience",
        expertise=["APIs", "Python", "System Architecture"],
        email="john@example.com"
    )

    webinar_id = system.create_webinar(
        content=content,
        webinar_type=WebinarType.TECHNICAL_DEEP_DIVE,
        start_time=datetime.now() + timedelta(days=7),
        duration_minutes=90,
        max_participants=100,
        presenters=[presenter]
    )

    # Register participants
    system.register_participant(
        webinar_id=webinar_id,
        email="participant@example.com",
        name="Jane Smith",
        company="Tech Corp",
        job_title="Developer"
    )

    # Generate report
    report = system.generate_webinar_report(webinar_id)
    print("Webinar Report:", report)

    # Clean up
    system.close()

if __name__ == "__main__":
    main()

Webinar Report: {'webinar_info': {'title': 'Advanced API Integration Techniques', 'type': 'Technical Deep Dive', 'status': 'Scheduled'}, 'participation': {'registered': 1, 'attended': 0, 'attendance_rate': 0.0}, 'feedback': {'average_rating': None, 'demographics': {'Tech Corp': 1}}}


In [ ]:
def get_webinar_recommendations(self, participant_email: str) -> List[Dict]:
    try:
        cursor = self.conn.cursor()

        # Get participant's interests and experience level
        cursor.execute('''
        SELECT experience_level, interests
        FROM participants
        WHERE email = ?
        ''', (participant_email,))

        participant_info = cursor.fetchone()
        if not participant_info:
            return []

        experience_level, interests = participant_info
        interests = json.loads(interests)

        # Find upcoming webinars matching participant's interests and level
        cursor.execute('''
        SELECT
            w.id,
            w.title,
            w.description,
            w.start_time,
            w.difficulty_level,
            w.tags
        FROM webinars w
        WHERE w.start_time > DATETIME('now')
        AND w.status = ?
        ''', (WebinarStatus.SCHEDULED.value,))

        webinars = cursor.fetchall()
        recommendations = []

        for webinar in webinars:
            tags = json.loads(webinar[5])
            relevance_score = len(set(tags) & set(interests)) / len(tags) if tags else 0

            if relevance_score > 0.3:  # Minimum relevance threshold
                recommendations.append({
                    'id': webinar[0],
                    'title': webinar[1],
                    'description': webinar[2],
                    'start_time': webinar[3],
                    'difficulty_level': webinar[4],
                    'relevance_score': relevance_score
                })  # Added closing brace here

        return recommendations

    except Exception as e:
        logger.error(f"Error getting webinar recommendations: {str(e)}")
        raise

In [ ]:
from datetime import datetime, timedelta
import sqlite3
from enum import Enum
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
import pandas as pd
import uuid
import pytz
from dataclasses import dataclass
from typing import List, Dict, Optional
import json
from pathlib import Path
import logging
from concurrent.futures import ThreadPoolExecutor

# Add logging configuration
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler('webinar_system.log'),
        logging.StreamHandler()
    ]
)
logger = logging.getLogger(__name__)

class WebinarStatus(Enum):
    SCHEDULED = "Scheduled"
    LIVE = "Live"
    COMPLETED = "Completed"
    CANCELLED = "Cancelled"
    RESCHEDULED = "Rescheduled"  # New status
    WAITLIST = "Waitlist"  # New status

class WebinarType(Enum):
    PRODUCT_DEMO = "Product Demo"
    TECHNICAL_DEEP_DIVE = "Technical Deep Dive"
    HANDS_ON_WORKSHOP = "Hands-on Workshop"
    Q_AND_A = "Q&A Session"
    FEATURE_SHOWCASE = "Feature Showcase"
    CUSTOMER_STORY = "Customer Story"  # New type
    CERTIFICATION_PREP = "Certification Preparation"  # New type

@dataclass
class Presenter:
    id: int
    name: str
    title: str
    bio: str
    expertise: List[str]
    email: str
    linkedin_profile: Optional[str] = None  # New field
    past_webinars: Optional[List[str]] = None  # New field

@dataclass
class WebinarContent:
    title: str
    description: str
    agenda: List[str]
    prerequisites: List[str]
    resources: Dict[str, str]
    presentation_url: Optional[str] = None
    recording_url: Optional[str] = None
    target_audience: Optional[str] = None  # New field
    difficulty_level: Optional[str] = None  # New field
    keywords: Optional[List[str]] = None  # New field

@dataclass
class WaitlistEntry:
    participant_email: str
    registration_date: datetime
    notification_sent: bool = False

class WebinarManagementSystem:
    def __init__(self, db_name="webinar_system.db", timezone="UTC"):
        self.conn = sqlite3.connect(db_name)
        self.timezone = pytz.timezone(timezone)
        self.waitlist: Dict[str, List[WaitlistEntry]] = {}
        self.setup_database()

    def setup_database(self):
        cursor = self.conn.cursor()

        # Adding new fields to existing tables
        cursor.execute('''ALTER TABLE webinars ADD COLUMN target_audience TEXT''')
        cursor.execute('''ALTER TABLE webinars ADD COLUMN difficulty_level TEXT''')
        cursor.execute('''ALTER TABLE webinars ADD COLUMN keywords TEXT''')
        cursor.execute('''ALTER TABLE webinars ADD COLUMN series_id TEXT''')

        # New table for series
        cursor.execute('''
        CREATE TABLE IF NOT EXISTS webinar_series (
            id TEXT PRIMARY KEY,
            name TEXT,
            description TEXT,
            total_sessions INTEGER
        )''')

        # New table for polls
        cursor.execute('''
        CREATE TABLE IF NOT EXISTS polls (
            id INTEGER PRIMARY KEY,
            webinar_id TEXT,
            question TEXT,
            options TEXT,
            created_at DATETIME,
            FOREIGN KEY (webinar_id) REFERENCES webinars (id)
        )''')

        # New table for poll responses
        cursor.execute('''
        CREATE TABLE IF NOT EXISTS poll_responses (
            id INTEGER PRIMARY KEY,
            poll_id INTEGER,
            participant_id INTEGER,
            response TEXT,
            submitted_at DATETIME,
            FOREIGN KEY (poll_id) REFERENCES polls (id),
            FOREIGN KEY (participant_id) REFERENCES participants (id)
        )''')

        # New table for certificates
        cursor.execute('''
        CREATE TABLE IF NOT EXISTS certificates (
            id INTEGER PRIMARY KEY,
            webinar_id TEXT,
            participant_id INTEGER,
            issued_date DATETIME,
            certificate_url TEXT,
            FOREIGN KEY (webinar_id) REFERENCES webinars (id),
            FOREIGN KEY (participant_id) REFERENCES participants (id)
        )''')

        self.conn.commit()

    def create_webinar_series(self, name: str, description: str, total_sessions: int) -> str:
        series_id = str(uuid.uuid4())
        cursor = self.conn.cursor()

        cursor.execute('''
        INSERT INTO webinar_series (id, name, description, total_sessions)
        VALUES (?, ?, ?, ?)
        ''', (series_id, name, description, total_sessions))

        self.conn.commit()
        return series_id

    def add_poll(self, webinar_id: str, question: str, options: List[str]) -> int:
        cursor = self.conn.cursor()

        cursor.execute('''
        INSERT INTO polls (webinar_id, question, options, created_at)
        VALUES (?, ?, ?, ?)
        ''', (webinar_id, question, json.dumps(options), datetime.now()))

        poll_id = cursor.lastrowid
        self.conn.commit()
        return poll_id

    def submit_poll_response(self, poll_id: int, participant_email: str, response: str):
        cursor = self.conn.cursor()

        cursor.execute('''
        INSERT INTO poll_responses (poll_id, participant_id, response, submitted_at)
        VALUES (?, (SELECT id FROM participants WHERE email = ?), ?, ?)
        ''', (poll_id, participant_email, response, datetime.now()))

        self.conn.commit()

    def generate_certificate(self, webinar_id: str, participant_email: str) -> str:
        # In a real implementation, this would generate a PDF certificate
        certificate_url = f"certificates/{webinar_id}/{participant_email}.pdf"

        cursor = self.conn.cursor()
        cursor.execute('''
        INSERT INTO certificates (webinar_id, participant_id, issued_date, certificate_url)
        VALUES (?, (SELECT id FROM participants WHERE email = ?), ?, ?)
        ''', (webinar_id, participant_email, datetime.now(), certificate_url))

        self.conn.commit()
        return certificate_url

    def add_to_waitlist(self, webinar_id: str, participant_email: str):
        if webinar_id not in self.waitlist:
            self.waitlist[webinar_id] = []

        entry = WaitlistEntry(
            participant_email=participant_email,
            registration_date=datetime.now()
        )
        self.waitlist[webinar_id].append(entry)
        logger.info(f"Added {participant_email} to waitlist for webinar {webinar_id}")

    def process_waitlist(self, webinar_id: str):
        if webinar_id not in self.waitlist:
            return

        cursor = self.conn.cursor()
        cursor.execute('''
        SELECT max_participants,
               (SELECT COUNT(*) FROM webinar_participants WHERE webinar_id = ?) as current_count
        FROM webinars
        WHERE id = ?
        ''', (webinar_id, webinar_id))

        result = cursor.fetchone()
        if not result:
            return

        max_participants, current_count = result
        available_spots = max_participants - current_count

        if available_spots <= 0:
            return

        # Process waitlist entries
        for entry in self.waitlist[webinar_id][:available_spots]:
            if self.register_participant(
                webinar_id,
                entry.participant_email,
                "Waitlist Registration",
                "From Waitlist",
                "Waitlisted Participant"
            ):
                entry.notification_sent = True
                logger.info(f"Processed waitlist entry for {entry.participant_email}")

    def export_webinar_data(self, webinar_id: str, format: str = 'csv') -> str:
        """Export webinar data in specified format (csv or json)"""
        data = self.generate_webinar_report(webinar_id)

        if format.lower() == 'csv':
            df = pd.DataFrame([data])
            output_file = f"webinar_{webinar_id}_report.csv"
            df.to_csv(output_file, index=False)
        else:
            output_file = f"webinar_{webinar_id}_report.json"
            with open(output_file, 'w') as f:
                json.dump(data, f, indent=4)

        return output_file

    def bulk_email_participants(self, webinar_id: str, subject: str, message: str):
        """Send bulk emails to all participants of a webinar"""
        cursor = self.conn.cursor()
        cursor.execute('''
        SELECT p.email
        FROM participants p
        JOIN webinar_participants wp ON p.id = wp.participant_id
        WHERE wp.webinar_id = ?
        ''', (webinar_id,))

        emails = [row[0] for row in cursor.fetchall()]

        with ThreadPoolExecutor(max_workers=5) as executor:
            executor.map(
                lambda email: self._send_email(email, subject, message),
                emails
            )

    def _send_email(self, email: str, subject: str, message: str):
        """Helper method to send individual emails"""
        try:
            # Email configuration would go here
            logger.info(f"Sent email to {email}: {subject}")
        except Exception as e:
            logger.error(f"Failed to send email to {email}: {str(e)}")

    def get_presenter_statistics(self, presenter_email: str) -> Dict:
        """Get statistics for a specific presenter"""
        cursor = self.conn.cursor()

        cursor.execute('''
        SELECT
            COUNT(DISTINCT w.id) as total_webinars,
            AVG(f.rating) as average_rating,
            SUM(CASE WHEN w.status = 'Completed' THEN 1 ELSE 0 END) as completed_webinars
        FROM presenters p
        JOIN webinar_presenters wp ON p.id = wp.presenter_id
        JOIN webinars w ON wp.webinar_id = w.id
        LEFT JOIN feedback f ON w.id = f.webinar_id
        WHERE p.email = ?
        ''', (presenter_email,))

        return dict(zip(
            ['total_webinars', 'average_rating', 'completed_webinars'],
            cursor.fetchone()
        ))

    def get_participant_engagement(self, participant_email: str) -> Dict:
        """Get engagement metrics for a specific participant"""
        cursor = self.conn.cursor()

        cursor.execute('''
        SELECT
            COUNT(DISTINCT wp.webinar_id) as registered_webinars,
            SUM(CASE WHEN wp.attended THEN 1 ELSE 0 END) as attended_webinars,
            COUNT(DISTINCT f.id) as feedback_submitted,
            COUNT(DISTINCT pr.id) as poll_responses
        FROM participants p
        LEFT JOIN webinar_participants wp ON p.id = wp.participant_id
        LEFT JOIN feedback f ON p.id = f.participant_id
        LEFT JOIN poll_responses pr ON p.id = pr.participant_id
        WHERE p.email = ?
        ''', (participant_email,))

        return dict(zip(
            ['registered_webinars', 'attended_webinars',
             'feedback_submitted', 'poll_responses'],
            cursor.fetchone()
        ))

# Example usage remains the same as before

In [ ]:
# First code cell - Create the necessary files
%%writefile webinar_system.py
# [Paste the entire code with imports, classes, and WebinarManagementSystem class here]

Writing webinar_system.py


In [ ]:
# Second code cell - Create the test file
%%writefile webinar_test.py
# [Paste the test code here]

Writing webinar_test.py


In [ ]:
# First cell - Install required packages
!pip install pytz

In [ ]:
# Second cell - Main system code
%%writefile webinar_system.py
from datetime import datetime, timedelta
import sqlite3
from enum import Enum
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
import pandas as pd
import uuid
import pytz
from dataclasses import dataclass
from typing import List, Dict, Optional
import logging
import json

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

# [Rest of your class definitions - WebinarStatus, WebinarType, Presenter, WebinarContent]
# [Complete WebinarManagementSystem class]

Overwriting webinar_system.py


In [ ]:
# Third cell - Test code
%%writefile test_webinar.py
from webinar_system import *

def test_webinar_system():
    print("\nStarting Webinar System Test...")

    try:
        # Initialize system
        print("\n1. Initializing system...")
        system = WebinarManagementSystem(db_name="test_webinar.db")
        print("✓ System initialized")

        # Create series
        print("\n2. Creating test series...")
        series_id = system.create_webinar_series(
            name="Test Series",
            description="Test Description",
            total_sessions=2
        )
        print(f"✓ Series created: {series_id}")

        # Create webinar content
        content = WebinarContent(
            title="Test Webinar",
            description="Test Description",
            agenda=["Item 1", "Item 2"],
            prerequisites=["None"],
            resources={},
            target_audience="All",
            difficulty_level="Beginner",
            keywords=["test"]
        )

        # Create presenter
        presenter = Presenter(
            id=1,
            name="Test Presenter",
            title="Tester",
            bio="Test Bio",
            expertise=["Testing"],
            email="test@example.com"
        )

        # Create webinar
        print("\n3. Creating test webinar...")
        webinar_id = system.create_webinar(
            content=content,
            webinar_type=WebinarType.TECHNICAL_DEEP_DIVE,
            start_time=datetime.now() + timedelta(days=1),
            duration_minutes=60,
            max_participants=10,
            presenters=[presenter]
        )
        print(f"✓ Webinar created: {webinar_id}")

        # Register participant
        print("\n4. Registering participant...")
        success = system.register_participant(
            webinar_id=webinar_id,
            email="test@participant.com",
            name="Test User",
            company="Test Co",
            job_title="Tester"
        )
        print(f"✓ Registration {'successful' if success else 'failed'}")

        # Clean up
        system.close()
        print("\n✓ Test completed successfully!")

    except Exception as e:
        print(f"\n❌ Error: {type(e).__name__} - {str(e)}")
        raise

if __name__ == "__main__":
    test_webinar_system()

Writing test_webinar.py


In [ ]:
# Fourth cell - Run the test
!python test_webinar.py

python3: can't open file '/content/test_webinar.py': [Errno 2] No such file or directory


In [ ]:
# Fifth cell - List files
!ls -l

total 80
drwxr-xr-x 2 root root  4096 Dec 29 04:19 __pycache__
drwxr-xr-x 1 root root  4096 Dec 19 14:20 sample_data
-rw-r--r-- 1 root root  2316 Dec 29 04:19 test_webinar.py
-rw-r--r-- 1 root root 57344 Dec 29 03:55 webinar_system.db
-rw-r--r-- 1 root root     0 Dec 29 03:53 webinar_system.log
-rw-r--r-- 1 root root   628 Dec 29 04:19 webinar_system.py
-rw-r--r-- 1 root root    29 Dec 29 04:17 webinar_test.py


In [ ]:
# Sixth cell - View database contents (optional)
import sqlite3

def view_database():
    conn = sqlite3.connect('test_webinar.db')
    cursor = conn.cursor()

    print("Tables in database:")
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    tables = cursor.fetchall()

    for table in tables:
        print(f"\nContents of {table[0]}:")
        cursor.execute(f"SELECT * FROM {table[0]};")
        print(cursor.fetchall())

    conn.close()

view_database()

Tables in database:


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from google.colab import auth
auth.authenticate_user()

In [ ]:
!pip install aiosmtplib
!pip install aiosqlite
!pip install pandas

In [ ]:
import sqlite3
from enum import Enum
from dataclasses import dataclass
from typing import List, Dict, Optional
from datetime import datetime, timedelta
import json
import uuid

class WebinarStatus(Enum):
    SCHEDULED = "Scheduled"
    LIVE = "Live"
    COMPLETED = "Completed"
    CANCELLED = "Cancelled"

class WebinarType(Enum):
    PRODUCT_DEMO = "Product Demo"
    TECHNICAL_DEEP_DIVE = "Technical Deep Dive"
    HANDS_ON_WORKSHOP = "Hands-on Workshop"
    Q_AND_A = "Q&A Session"
    FEATURE_SHOWCASE = "Feature Showcase"

@dataclass
class Presenter:
    id: int
    name: str
    title: str
    bio: str
    expertise: List[str]
    email: str

@dataclass
class WebinarContent:
    title: str
    description: str
    agenda: List[str]
    prerequisites: List[str]
    resources: Dict[str, str]
    presentation_url: Optional[str] = None
    recording_url: Optional[str] = None

class WebinarManagementSystem:
    def __init__(self, db_name=":memory:"):
        self.conn = sqlite3.connect(db_name)
        self.setup_database()

    def setup_database(self):
        cursor = self.conn.cursor()

        cursor.execute('''
        CREATE TABLE IF NOT EXISTS webinars (
            id TEXT PRIMARY KEY,
            title TEXT,
            type TEXT,
            description TEXT,
            start_time DATETIME,
            duration_minutes INTEGER,
            max_participants INTEGER,
            status TEXT
        )''')

        cursor.execute('''
        CREATE TABLE IF NOT EXISTS participants (
            id INTEGER PRIMARY KEY,
            email TEXT UNIQUE,
            name TEXT,
            company TEXT,
            job_title TEXT
        )''')

        cursor.execute('''
        CREATE TABLE IF NOT EXISTS webinar_participants (
            webinar_id TEXT,
            participant_id INTEGER,
            registration_date DATETIME,
            FOREIGN KEY (webinar_id) REFERENCES webinars (id),
            FOREIGN KEY (participant_id) REFERENCES participants (id)
        )''')

        self.conn.commit()

    def create_webinar(self, content: WebinarContent, webinar_type: WebinarType,
                      start_time: datetime, duration_minutes: int,
                      max_participants: int, presenters: List[Presenter]) -> str:
        webinar_id = str(uuid.uuid4())
        cursor = self.conn.cursor()

        cursor.execute('''
        INSERT INTO webinars (
            id, title, type, description, start_time,
            duration_minutes, max_participants, status
        ) VALUES (?, ?, ?, ?, ?, ?, ?, ?)
        ''', (webinar_id, content.title, webinar_type.value, content.description,
              start_time, duration_minutes, max_participants,
              WebinarStatus.SCHEDULED.value))

        self.conn.commit()
        return webinar_id

    def register_participant(self, webinar_id: str, email: str, name: str,
                           company: str, job_title: str) -> bool:
        cursor = self.conn.cursor()

        # Insert participant
        cursor.execute('''
        INSERT OR IGNORE INTO participants (email, name, company, job_title)
        VALUES (?, ?, ?, ?)
        ''', (email, name, company, job_title))

        # Register for webinar
        cursor.execute('''
        INSERT INTO webinar_participants (webinar_id, participant_id, registration_date)
        VALUES (?, (SELECT id FROM participants WHERE email = ?), ?)
        ''', (webinar_id, email, datetime.now()))

        self.conn.commit()
        return True

    def generate_webinar_report(self, webinar_id: str) -> Dict:
        cursor = self.conn.cursor()

        cursor.execute('''
        SELECT w.*, COUNT(wp.participant_id) as registered_participants
        FROM webinars w
        LEFT JOIN webinar_participants wp ON w.id = wp.webinar_id
        WHERE w.id = ?
        GROUP BY w.id
        ''', (webinar_id,))

        webinar = cursor.fetchone()
        if not webinar:
            return {}

        return {
            'webinar_info': {
                'id': webinar[0],
                'title': webinar[1],
                'type': webinar[2],
                'status': webinar[7]
            },
            'participants': {
                'registered': webinar[8]
            }
        }

    def close(self):
        self.conn.close()

# Demo usage
async def demo_webinar_system():
    system = WebinarManagementSystem()

    content = WebinarContent(
        title="Python for Data Science",
        description="Learn Python basics for DS",
        agenda=["Intro", "Pandas", "NumPy"],
        prerequisites=["Basic Programming"],
        resources={"Guide": "https://example.com"}
    )

    presenter = Presenter(
        id=1,
        name="Jane Smith",
        title="Data Scientist",
        bio="10 years experience",
        expertise=["Python", "ML"],
        email="jane@example.com"
    )

    # Create webinar
    webinar_id = system.create_webinar(
        content=content,
        webinar_type=WebinarType.HANDS_ON_WORKSHOP,
        start_time=datetime.now() + timedelta(days=1),
        duration_minutes=120,
        max_participants=50,
        presenters=[presenter]
    )

    print(f"Created webinar with ID: {webinar_id}")

    # Register participants
    participants = [
        ("user1@test.com", "User One", "TechCorp", "Developer"),
        ("user2@test.com", "User Two", "DataCo", "Analyst"),
        ("user3@test.com", "User Three", "AILabs", "Researcher")
    ]

    for email, name, company, title in participants:
        system.register_participant(webinar_id, email, name, company, title)
        print(f"Registered participant: {name}")

    # Generate report
    report = system.generate_webinar_report(webinar_id)
    print("\nWebinar Report:")
    print(json.dumps(report, indent=2))

    system.close()

# Run the demo
await demo_webinar_system()

Created webinar with ID: c0678721-efed-40d3-a860-ffcd712a3cf0
Registered participant: User One
Registered participant: User Two
Registered participant: User Three

Webinar Report:
{
  "webinar_info": {
    "id": "c0678721-efed-40d3-a860-ffcd712a3cf0",
    "title": "Python for Data Science",
    "type": "Hands-on Workshop",
    "status": "Scheduled"
  },
  "participants": {
    "registered": 3
  }
}
